In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
def non_shuffling_train_test_split(X, y, test_size=0.2):
    i = int((1 - test_size) * X.shape[0]) + 1
    X_train, X_test = np.split(X, [i])
    y_train, y_test = np.split(y, [i])
    return X_train, X_test, y_train, y_test

In [3]:
def compress_rows(df_seq):
    data = {}
    for game in df_seq.index.unique():
        data[game] = []
    for game in tqdm(df_seq.index.unique()):
        data[game].append(df_seq[df_seq.columns].loc[game].to_numpy())
    return data

In [25]:
def split_data(df, test_size=0.2):
    games = list(df['Game_ID'].unique())
    count = len(games)
    i = int((1 - test_size) * count)
    train_ids, test_ids = games[:i], games[i:]
    train = df[df['Game_ID'].isin(train_ids)]
    test = df[df['Game_ID'].isin(test_ids)]
    train.drop('Game_ID', axis = 1, inplace=True)
    test.drop('Game_ID', axis = 1, inplace=True)
    return train, test

In [26]:
df = pd.read_csv("../Data/single_dataset.csv", low_memory=False)
df.dropna(axis=0, how='any', inplace=True)

#enc characters
enc_p1 = LabelEncoder()
enc_p2 = LabelEncoder()

enc_p1.fit(df['CHAR_P1'])
enc_p2.fit(df['CHAR_P2'])
df['CHAR_P1'] = enc_p1.transform(df['CHAR_P1'])
df['CHAR_P2'] = enc_p2.transform(df['CHAR_P2'])

X_cols = ['CHAR_P1', 'CHAR_P2', 'S_airborne_P1',
   'S_damage_P1', 'S_direction_P1', 'S_hit_stun_P1', 'S_position_x_P1',
   'S_position_y_P1', 'S_shield_P1', 'S_state_P1', 'S_state_age_P1',
   'S_stocks_P1', 'S_airborne_P2', 'S_damage_P2', 'S_direction_P2',
   'S_hit_stun_P2', 'S_position_x_P2', 'S_position_y_P2', 'S_shield_P2',
   'S_state_P2', 'S_state_age_P2', 'S_stocks_P2']

y_cols = ['B_damage_P1',
   'B_direction_P1', 'B_joystick_x_P1', 'B_joystick_y_P1',
   'B_position_x_P1', 'B_position_y_P1', 'B_cstick_x_P1', 'B_cstick_y_P1',
   'B_state_P1', 'B_raw_analog_P1', 'B_buttons_physical_P1',
   'B_triggers_physical_l_P1', 'B_triggers_physical_r_P1',
   'B_triggers_logical_P1']

df['S_airborne_P1'] = df['S_airborne_P1'].astype(bool).astype(float)
df['S_airborne_P2'] = df['S_airborne_P2'].astype(bool).astype(float)

train, test = split_data(df)

C:\Users\steph\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [30]:
train_scaler = StandardScaler().fit(train)
train = train_scaler.transform(train)
test = train_scaler.transform(test)

In [31]:
type(train)

numpy.ndarray

In [117]:
# X_train, X_test, y_train, y_test = non_shuffling_train_test_split(X, y)

In [114]:
# X_scaler = StandardScaler().fit(train)
# y_scaler = StandardScaler().fit(y_train)

# X_train = X_scaler.transform(X_train)
# X_test = X_scaler.transform(X_test)


# y_train = y_scaler.transform(y_train)
# y_test = y_scaler.transform(y_test)

In [119]:
datasets = [X_train, X_test, y_train, y_test]

datasets_seq = []

for d in datasets:
    datasets_seq.append(compress_rows(d))

100%|██████████| 4/4 [00:00<00:00, 399.91it/s]


In [121]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [162]:
model.initHidden().shape

torch.Size([1, 128])

In [122]:
input_size = X_train.shape[1]
output_size = y_train.shape[1]
hidden_size = 128

epochs = 10

model = RNN(input_size, n_hidden, output_size)

learning_rate = 0.005

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  
criterion = nn.MSELoss()

In [186]:
def t(a):
    return torch.tensor(a,dtype=torch.float32)

def train(x_train, y_train, replay):
    hidden = model.initHidden()

    model.zero_grad()
    
    replay_in = x_train[replay][0]
    replay_out = y_train[replay][0]
    predicted = [] 
    actual = []
    
    for i in tqdm(range(replay_in.shape[0])):
        
        output, hidden = model(t([replay_in[i]]), hidden)

        
        #loss = criterion(output, t([replay_out[i]]))
        
    loss.backward()

    optimizer.step()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in model.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return loss.item()

In [173]:
 torch.autograd.set_detect_anomaly(True)

In [187]:
X_train = datasets_seq[0]
y_train = datasets_seq[2]

for epoch in range(epochs):
    for name in X_train.keys():
        loss = train(X_train, y_train, name)
        train_losses.append(loss)

    print('epoch {}, train loss {}'.format(epoch, np.average(train_losses)))
    print(' ')
        

  5%|▌         | 1067/20858 [00:02<00:55, 357.45it/s]


KeyboardInterrupt: 